In [1]:
%run Requirements.ipynb

24/05/23 23:48:18 WARN Utils: Your hostname, Ronits-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.2 instead (on interface en0)
24/05/23 23:48:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/23 23:48:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
%run secrets.ipynb

In [3]:
combined_df = spark.sql(
    """
    SELECT DISTINCT
        ART.ArtistName,
        ART.Listeners AS ArtistListeners,
        ART.PlayCount AS ArtistPlayCount,
        ART.Genres AS ArtistGenres,
        TRA.TrackName,
        TRA.Duration AS TrackLength,
        TRA.Listeners AS TrackListeners,
        TRA.PlayCount AS TrackPlayCount,
        ALB.AlbumName,
        ALB.Listeners AS AlbumListeners,
        ALB.PlayCount AS AlbumPlayCount,
        ALB.ReleaseDate AS AlbumReleaseDate
    FROM
        PARQUET.`Mart/Artists_LastFM` AS ART
    JOIN
        PARQUET.`Mart/Tracks_LastFM` AS TRA ON LTRIM(RTRIM(LOWER(ART.ArtistName))) = LTRIM(RTRIM(LOWER(TRA.ArtistName)))
    JOIN
        PARQUET.`Mart/Albums_LastFM` AS ALB ON LTRIM(RTRIM(LOWER(ALB.ArtistName))) = LTRIM(RTRIM(LOWER(TRA.ArtistName)))
    """
)

combined_df.write.mode('overwrite').format('parquet').save('Mart/GlobalArtist_LastFMDetails')


In [4]:
combined_df = spark.read.parquet("Mart/GlobalArtist_LastFMDetails")
combined_pd_df = combined_df.toPandas()
spotify_token = AT
def get_spotify_artist_info(artist_name, access_token):
    search_url = f"https://api.spotify.com/v1/search?q={artist_name}&type=artist"
    headers = {"Authorization": f"Bearer {access_token}"}
    response = requests.get(search_url, headers=headers)
    data = response.json()
    if data['artists']['items']:
        artist = data['artists']['items'][0]
        return {
            "ArtistPopularity": artist.get('popularity'),
            "ArtistFollowers": artist['followers'].get('total')
        }
    return {"ArtistPopularity": None, "ArtistFollowers": None}

def get_spotify_track_info(track_name, artist_name, access_token):
    search_url = f"https://api.spotify.com/v1/search?q=track:{track_name} artist:{artist_name}&type=track"
    headers = {"Authorization": f"Bearer {access_token}"}
    response = requests.get(search_url, headers=headers)
    data = response.json()
    if data['tracks']['items']:
        track = data['tracks']['items'][0]
        return {
            "TrackPopularity": track.get('popularity'),
            "TrackPreview": track.get('preview_url')
        }
    return {"TrackPopularity": None, "TrackPreview": None}

def get_spotify_album_info(album_name, artist_name, access_token):
    search_url = f"https://api.spotify.com/v1/search?q=album:{album_name} artist:{artist_name}&type=album"
    headers = {"Authorization": f"Bearer {access_token}"}
    response = requests.get(search_url, headers=headers)
    data = response.json()
    if data['albums']['items']:
        album = data['albums']['items'][0]
        return {
            "AlbumTotalTracks": album.get('total_tracks')
        }
    return {"AlbumTotalTracks": None}

enhanced_data = []

for index, row in combined_pd_df.iterrows():
    artist_name = row["ArtistName"]
    track_name = row["TrackName"]
    album_name = row["AlbumName"]

    artist_info = get_spotify_artist_info(artist_name, spotify_token)
    track_info = get_spotify_track_info(track_name, artist_name, spotify_token)
    album_info = get_spotify_album_info(album_name, artist_name, spotify_token)

    enhanced_info = {**row, **artist_info, **track_info, **album_info}

    enhanced_data.append(enhanced_info)

enhanced_df = pd.DataFrame(enhanced_data)

enhanced_spark_df = spark.createDataFrame(enhanced_df)
enhanced_spark_df.write.mode('overwrite').format('parquet').save('Mart/GlobalArtist_SpotifyDetails')


24/05/23 23:49:44 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
